In [1]:
import http.server
import socketserver
import json
import time
import os.path
import Crypto
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto import Random
import secrets
import base64
import hashlib
import uuid
import random
import string
import traceback
import smtplib
from email.message import EmailMessage

In [2]:
current_time_millis = lambda: int(round(time.time() * 1000))

def random_string():
    return ''.join(random.choice(string.ascii_lowercase + string.ascii_uppercase + string.digits) for i in range(random.randint(20, 40)))

In [3]:
path_public_rsa = 'public_rsa'
path_get_file = 'get_file'
path_session_key = 'session_key'
path_login = 'login'
path_send_file = 'send_file'
path_logout = 'logout'
key_public_exponent = 'exponent'
key_modulus = 'modulus'
key_filename = 'file'
key_username = 'username'
key_password = 'password'
key_error = 'error'
key_content = 'content'
key_iv = 'iv'
json_key_filename = 'filename'
json_key_file_content = 'file_content'
key_extra_symbols = 'extra_symbols'
session_key_expiration_period = 1 * 60 * 1000 # 1 minute
public_key_filename = 'public_key.txt'
key_token = 'token'
key_request_code = 'request_code'
key_ver_code = 'ver_code'
path_ver_code = 'verification'
key_email = 'email'
key_result = 'result'
key_session_id = 'session_id'

class SessionKey:
    
    def __init__(self, key):
        self.key = key
        self.created = current_time_millis()
    
    def is_expired(self):
        return current_time_millis() - self.created > session_key_expiration_period
    
class DataHolder:
    
    def __init__(self):
        self.session_key = None
        self.public_exponent = None
        self.modulus = None
        self.secret_request_code = None
        self.ver_code = None
        self.session_id = None
        
    def check_username_and_password(self, username, password_json):
        print("check_username_and_password, session key: ", str(base64.b64encode(self.session_key.key)))
        if not os.path.isfile(username + '.txt'):
            return 'WRONG_USERNAME'
        if self.session_key is None:
            return 'NO_SESSION_KEY'
        try:
            decrypted_password = self.decrypt_aes(password_json)
            print('password:', decrypted_password)
            decrypted_password_hash = hashlib.md5(decrypted_password).hexdigest()
            file = open(username + '.txt', 'r')
            user_json = json.loads(file.read())
            actual_password_hash = user_json[key_password]
            file.close()
            if decrypted_password_hash != actual_password_hash:
                return 'WRONG_PASSWORD'
            else:
                return 'OK'
        except Exception as e:
            print(e)
            traceback.print_exc()
        return 'ERROR'
    
    def get_email(self, username):
        try:
            file = open(username + '.txt', 'r')
            user_json = json.loads(file.read())
            email = user_json[key_email]
            file.close()
            return email
        except Exception as e:
            print(e)
            traceback.print_exec()
        return None

    def get_string_from_bytes(self, bytes_arr):
        return str(base64.b64encode(bytes_arr), "utf-8")
    
    def get_file_response(self, post_body, filename):
        response = {}
        print("get_file_response, session key: ", str(base64.b64encode(self.session_key.key)))
        decrypted_body = self.decrypt_aes(post_body)
        decrypted_body_json = json.loads(decrypted_body)
        client_session_id = decrypted_body_json[key_session_id]
        if (client_session_id != self.session_id):
            print('expected session id: ', self.session_id, 'actual: ', client_session_id)
            response[key_error] = 'authorization_error'
        else:
            if self.session_key is None or self.session_key.is_expired():
                response[key_error] = 'session_key_expired'
            elif not os.path.isfile(filename):
                response[key_error] = 'no_file'
            else:            
                file = open(filename, "r")
                file_content = file.read()
                file.close()            
                response[json_key_file_content] = self.encrypt_aes(file_content)    
        return json.dumps(response)
    
    def handle_send_file(self, post_json):
        try:
            decrypted_json = json.loads(self.decrypt_aes(post_json))
            filename = decrypted_json[json_key_filename]
            request_code = decrypted_json[key_request_code]
            content = decrypted_json[json_key_file_content]
            client_session_id = decrypted_json[key_session_id]
            if client_session_id != self.session_id:
                return (404, '')
            if self.secret_request_code is not None:
                if request_code != self.secret_request_code:
                    return (404, '')
            file = open(filename, "w+")
            file.write(content)
            file.close()
            self.secret_request_code = random_string()
            return (200, json.dumps(self.encrypt_aes(self.secret_request_code)))
        except Exception as e:
            print(e)
            traceback.print_exc()
        return (413, '')
    
    def logout(self, post_json):
        decrypted_body_json = json.loads(self.decrypt_aes(post_json))
        client_session_id = decrypted_body_json[key_session_id]        
        if (client_session_id != self.session_id):
            return 401
        else:
            self.session_is = None
            return 200
    
    def decrypt_aes(self, encrypted_json):
        iv = base64.b64decode(bytes(encrypted_json[key_iv], 'utf-8'))
        extra_symbols = encrypted_json[key_extra_symbols]
        encrypted_text = base64.b64decode(bytes(encrypted_json[key_content], 'utf-8'))
        aes = AES.new(self.session_key.key, AES.MODE_OFB, iv)
        return aes.decrypt(encrypted_text)[:-extra_symbols]
    
    def encrypt_aes(self, decrypted_content):
        result_json = {}

        decrypted_bytes = bytes(decrypted_content, 'utf-8')
        extra_symbols = 16 - len(decrypted_bytes) % 16
        decrypted_bytes += bytes('0' * extra_symbols, 'utf-8')
        iv = Random.new().read(16)
        aes = AES.new(self.session_key.key, AES.MODE_OFB, iv)

        result_json[key_content] = self.get_string_from_bytes(aes.encrypt(decrypted_bytes))
        result_json[key_iv] = self.get_string_from_bytes(iv)
        result_json[key_extra_symbols] = extra_symbols
        return result_json

In [4]:
clients = {}

class MyHTTPRequestHandler(http.server.BaseHTTPRequestHandler):

    def do_GET(self):
        split_by_question = self.path.split('?')
        path = split_by_question[0]
        argument_dict = {}
        if len(split_by_question) > 1:
            argument_pairs = split_by_question[1].split('&')
            for pair in argument_pairs:
                key_and_value = pair.split('=')
                argument_dict[key_and_value[0]] = key_and_value[1]
        if (path[0] == '/'):
            path = path[1:]
        if path[-1] == '/':
            path = path[:-1]
        if path == path_session_key:
            self.handle_session_key(argument_dict)
        else:
            self.send_response(404)
            self.end_headers()            

    def do_POST(self):
        split_by_question = self.path.split('?')
        path = split_by_question[0]
        if (path[0] == '/'):
            path = path[1:]
        if path[-1] == '/':
            path = path[:-1]
        content_len = int(self.headers.get('Content-Length'))
        post_body = self.rfile.read(content_len)
        argument_dict = {}
        if len(split_by_question) > 1:
            argument_pairs = split_by_question[1].split('&')
            for pair in argument_pairs:
                key_and_value = pair.split('=')
                argument_dict[key_and_value[0]] = key_and_value[1]        
        if path == path_login:
            self.handle_login(post_body, argument_dict)
        elif path == path_public_rsa:
            self.handle_public_rsa(post_body)
        elif path == path_send_file:
            self.handle_send_file(post_body, argument_dict)
        elif path == path_ver_code:
            self.handle_ver_code(post_body, argument_dict)
        elif path == path_get_file:
            self.handle_get_file(post_body, argument_dict)
        elif path == path_logout:
            self.handle_logout(post_body, argument_dict)
        else:
            self.send_response(404)
            self.end_headers()            

    def handle_logout(self, post_body, arguments):
        try:
            data_holder = clients[arguments[key_token]]
            post_json = json.loads(post_body)
            code = data_holder.logout(post_json)
            self.send_response(code)
            self.end_headers()    
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers()    
            
    def handle_login(self, post_body, arguments):
        print('received post:', post_body)
        try:
            data_holder = clients[arguments[key_token]]
            post_json = json.loads(post_body)
            username = post_json[key_username]
            password_json = json.loads(post_json[key_password])
            result = data_holder.check_username_and_password(username, password_json)
            if (result == 'OK'):
                data_holder.ver_code = random.randrange(100000, 999999)
                email = data_holder.get_email(username)
                if email is None:
                    print('Could not get email for user', username)
                else:
                    self.send_email(email, data_holder.ver_code)
            print('username:', username)
            self.send_response(200)
            self.end_headers()            
            self.wfile.write(bytearray(result, 'utf8'))
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers()
            
    def handle_ver_code(self, post_body, arguments):
        try:
            data_holder = clients[arguments[key_token]]
            post_json = json.loads(post_body)
            encrypted_ver_code = post_json[key_ver_code]
            ver_code = int(data_holder.decrypt_aes(encrypted_ver_code).decode('utf-8'))
            response = {}
            if (ver_code == data_holder.ver_code):
                result = 'OK'
                session_id = str(uuid.uuid1())
                data_holder.session_id = session_id
                response[key_session_id] = data_holder.encrypt_aes(session_id)
            else:
                result = 'WRONG_VERIFICATION_CODE'
            response[key_result] = result
            self.send_response(200)
            self.end_headers()            
            self.wfile.write(bytearray(json.dumps(response), 'utf8'))
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers()
            
    def send_email(self, username, code):
        gmail_user = 'alive.qwerty@gmail.com'
        gmail_password = 'kmegxpvtbgpcunix'

        sent_from = gmail_user
        to = [username]
        subject = 'Your verification code'
        body = f"Verification code: {code}"

        email_text = """\
        From: %s
        To: %s
        Subject: %s

        %s
        """ % (sent_from, ", ".join(to), subject, body)

        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            server.sendmail(sent_from, to, email_text)
            server.close()

            print('Email sent!')
        except:
            print('Something went wrong...')        

    def handle_public_rsa(self, post_body):
        try:
            post_json = json.loads(post_body)
            data_holder = DataHolder()
            data_holder.public_exponent = int(post_json[key_public_exponent])
            data_holder.modulus = int(post_json[key_modulus])
            token = str(uuid.uuid1())
            clients[token] = data_holder
            print('handle_public_rsa: exp:', data_holder.public_exponent, ', modulus:', data_holder.modulus)
            self.send_response(200)
            self.end_headers()     
            self.wfile.write(bytearray(token, 'utf8'))
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers()     

    def handle_get_file(self, post_body, arguments):
        try:
            cur_token = arguments[key_token]
            print("handle_get_file, token: ", cur_token, "has client? ", (cur_token in clients))
            print('post_body:', str(post_body))
            data_holder = clients[arguments[key_token]]
            filename = arguments[key_filename]
            print('handle_get_file: ', filename)
            response = data_holder.get_file_response(json.loads(post_body), filename)
            self.send_response(200)
            self.end_headers()    
            self.wfile.write(bytes(response, 'utf-8'))
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers()    

    def handle_session_key(self, arguments):
        print('handle_session_key')
        try:
            token = arguments[key_token]
            print('token:', token)
            data_holder = clients[token]
            print('modulus:', data_holder.modulus, 'exponent:', data_holder.public_exponent)
            session_key = secrets.token_bytes(32)
            print("session_key:", str(base64.b64encode(session_key)))
            data_holder.session_key = SessionKey(session_key)
            public_key = RSA.construct((data_holder.modulus, data_holder.public_exponent))
            encrypted_session_key = public_key.encrypt(session_key, None)
            print("encrypted session key: ", str(base64.b64encode(encrypted_session_key[0])))

            self.send_response(200)
            self.end_headers()    
            self.wfile.write(encrypted_session_key[0])
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers() 
        
    def handle_send_file(self, post_body, arguments):
        try:
            data_holder = clients[arguments[key_token]]
            post_json = json.loads(post_body)
            (response, new_request_code) = data_holder.handle_send_file(post_json)
            self.send_response(response)
            self.end_headers()    
            self.wfile.write(bytes(new_request_code, 'utf-8'))
        except Exception as e:
            print(e)
            traceback.print_exc()
            self.send_response(413)
            self.end_headers()             

In [ ]:
port = 8000
Handler = http.server.SimpleHTTPRequestHandler

httpd = http.server.HTTPServer(('localhost', port), MyHTTPRequestHandler)
print("serving at port", port)
httpd.serve_forever()

serving at port 8000
handle_public_rsa: exp: 65537 , modulus: 7525440995132303755636096189812128851604588520186439703642628762158223153170265222811565308417518186657404911317414066944292309509956437484604037054850183


127.0.0.1 - - [25/Nov/2019 23:08:24] "POST /public_rsa HTTP/1.1" 200 -


handle_session_key
token: 55dcedac-0fbf-11ea-b372-acde48001122
modulus: 7525440995132303755636096189812128851604588520186439703642628762158223153170265222811565308417518186657404911317414066944292309509956437484604037054850183 exponent: 65537
session_key: b'e77LwXASCEYyP0YvQ4bFtdKva0mijOED5r5FEBB9cY4='
encrypted session key:  b'QkObvt8jEZxLFBm4K+kYuqWPEPDmKBTi35wIuQiqsct2+tIFgVpHqcrw87fN0JSgLMUPs6p0FswW1yJPNgq3Bw=='


127.0.0.1 - - [25/Nov/2019 23:08:24] "GET /session_key/?token=55dcedac-0fbf-11ea-b372-acde48001122 HTTP/1.1" 200 -


received post: b'{"username":"dzina","password":"{\\"content\\":\\"O1wBbs\\\\\\/NhDbrmh0VxapgcQ==\\\\n\\",\\"iv\\":\\"3\\\\\\/ZfYVbzdpBWkZOWXXiiLQ==\\\\n\\",\\"extra_symbols\\":10}"}'
check_username_and_password, session key:  b'e77LwXASCEYyP0YvQ4bFtdKva0mijOED5r5FEBB9cY4='
password: b'ddzina'
Email sent!
username: dzina


...

127.0.0.1 - - [25/Nov/2019 23:11:14] "POST /get_file?file=part3.txt&token=55dcedac-0fbf-11ea-b372-acde48001122 HTTP/1.1" 200 -


handle_session_key
token: 55dcedac-0fbf-11ea-b372-acde48001122
modulus: 7525440995132303755636096189812128851604588520186439703642628762158223153170265222811565308417518186657404911317414066944292309509956437484604037054850183 exponent: 65537
session_key: b'vIIn/nX2bu07MmI8lYsHrP3zWmSVLWzJsQ/pI3k9a9I='
encrypted session key:  b'XC512/0mYG+PzhGx9xjnDqJkVWUbYMJKVu8JaXwdRqYg63KkXuu8DEWONZmIko1d4QWEzsGcAeacKHv082YPDA=='


127.0.0.1 - - [25/Nov/2019 23:11:17] "GET /session_key/?token=55dcedac-0fbf-11ea-b372-acde48001122 HTTP/1.1" 200 -


handle_get_file, token:  55dcedac-0fbf-11ea-b372-acde48001122 has client?  True
post_body: b'{"content":"NQal2c+8BpdzGnYVtZoTtLXAe5l0A6FSqBvuHVELiWuYZYKftmolnMM1gEdKUjsbLhWo\\/EWKLN3m\\nb3rQ8hs1mA==\\n","iv":"lq3suKXtEQTA2hpqpvx08g==\\n","extra_symbols":11}'
handle_get_file:  part3.txt
get_file_response, session key:  b'vIIn/nX2bu07MmI8lYsHrP3zWmSVLWzJsQ/pI3k9a9I='


127.0.0.1 - - [25/Nov/2019 23:11:23] "POST /get_file?file=part3.txt&token=55dcedac-0fbf-11ea-b372-acde48001122 HTTP/1.1" 200 -
